# Coursera Capstone Project
#### The battle of the neighborhoods


* **Author: Alejandro (Alex) Vidaurri**
* **Date: May 10th 2021**

In this notebook we'll describe the battle of the neighborhoods using Location data and Data Science

In [1]:
import pandas as pd
import numpy as np


In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


In [18]:
import requests 

# Toronto Neighborhood Analysis

### Scraping data from 
*https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*

In [54]:
import requests
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

response = requests.get(url)

In [55]:
soup = BeautifulSoup(response.text)

In [62]:
table = soup.find(name='table')
table_contents = []
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

## Shape of Toronto Posta Codes 

In [79]:
df.shape

(103, 3)

## Adding Spatial Coordinates

In [86]:
tmp_df = pd.read_csv('./Geospatial_Coordinates.csv')

### Merging by postal code and dropping duplicate column

In [110]:
neighborhoods= pd.merge(left=df, right=tmp_df, how='inner', left_on='PostalCode', right_on='Postal Code')


In [113]:
neighborhoods.drop('Postal Code', axis=1, inplace=True)

In [114]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


## Canada M postal Codes map 

In [199]:
latitude=neighborhoods['Latitude'].mean()
longitude = neighborhoods['Longitude'].mean()
import folium # map rendering library

# create map of New York using latitude and longitude values
Canada = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Canada)  
    
Canada

### Toronto Neighborhoods

In [155]:
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
display(Toronto_data.head())
Toronto_data.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


(39, 5)

In [156]:
Toronto = folium.Map(location=[Toronto_data['Latitude'].mean(), Toronto_data['Longitude'].mean()], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto)  
    
Toronto

In [194]:
LAT_LON = '{},{}'.format(Toronto_data['Latitude'].mean(), Toronto_data['Longitude'].mean())
import urllib
RETURN_TYPE = 'json'
API_KEY= 'REMOVED'
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/{}?location={}&radius={}&key={}'.format(
    RETURN_TYPE,  LAT_LON, 500, API_KEY
)

print(url)

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=43.66772589743589,-79.38855562564103&radius=500&key=REMOVED


In [191]:
import pandas as pd
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lon in zip(names, latitudes, longitudes):
            
        # create the API request URL
        #url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        #    CLIENT_ID, 
        #    CLIENT_SECRET, 
        #    VERSION, 
        #    lat, 
        #    lng, 
        #    radius, 
        #    LIMIT)
        url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/{}?location={},{}&radius={}&key={}'.format(
            RETURN_TYPE,  lat, lon, radius, API_KEY
            )
         
        # make the GET request
        results = requests.get(url).json()['results']
        #results = pd.json_normalize(results) # flatten JSON
        #print( v['geometry.location.lat'], v['geometry.location.lat'])
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['geometry']['location']['lat'], 
            v['geometry']['location']['lng'],  
            v['types'][0]) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [192]:
Toronto_venues = getNearbyVenues(Toronto_data.loc[:, 'Neighborhood'],
                   Toronto_data.loc[:, 'Latitude'],
                    Toronto_data.loc[:, 'Longitude']
                   )

In [162]:
print(Toronto_venues.shape)
Toronto_venues.head()

(780, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.321558,Toronto,43.653226,-79.383184,locality
1,"Regent Park, Harbourfront",43.65426,-79.321558,Vistek Toronto,43.656842,-79.359975,electronics_store
2,"Regent Park, Harbourfront",43.65426,-79.321558,Porsche Centre Downtown Toronto,43.651916,-79.363160,finance
3,"Regent Park, Harbourfront",43.65426,-79.321558,Staples,43.651772,-79.363615,electronics_store
4,"Regent Park, Harbourfront",43.65426,-79.321558,Brick Street Bakery,43.650548,-79.359525,bakery


In [163]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,20,20,20,20,20,20
"Brockton, Parkdale Village, Exhibition Place",20,20,20,20,20,20
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",20,20,20,20,20,20
Central Bay Street,20,20,20,20,20,20
Christie,20,20,20,20,20,20
Church and Wellesley,20,20,20,20,20,20
"Commerce Court, Victoria Hotel",20,20,20,20,20,20
Davisville,20,20,20,20,20,20
Davisville North,20,20,20,20,20,20


In [164]:
# one hot encoding
toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,accounting,airport,art_gallery,bakery,bank,bar,beauty_salon,bicycle_store,book_store,...,spa,stadium,storage,store,sublocality_level_1,supermarket,tourist_attraction,transit_station,travel_agency,university
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,accounting,airport,art_gallery,bakery,bank,bar,beauty_salon,bicycle_store,book_store,...,spa,stadium,storage,store,sublocality_level_1,supermarket,tourist_attraction,transit_station,travel_agency,university
0,Berczy Park,0.00,0.0,0.00,0.00,0.00,0.15,0.00,0.00,0.00,...,0.00,0.05,0.00,0.00,0.00,0.00,0.05,0.00,0.05,0.00
1,"Brockton, Parkdale Village, Exhibition Place",0.05,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.00,0.2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.05,0.10,0.05
3,Central Bay Street,0.00,0.0,0.00,0.00,0.10,0.05,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00
4,Christie,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00
5,Church and Wellesley,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,"Commerce Court, Victoria Hotel",0.00,0.0,0.00,0.00,0.00,0.10,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
7,Davisville,0.00,0.0,0.05,0.05,0.00,0.05,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00
8,Davisville North,0.10,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00
9,"Dufferin, Dovercourt Village",0.00,0.0,0.05,0.00,0.00,0.05,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.05,0.00,0.00,0.00,0.00,0.00


In [171]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [181]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,restaurant,bar,lodging,night_club,neighborhood,tourist_attraction,museum,travel_agency,stadium,point_of_interest
1,"Brockton, Parkdale Village, Exhibition Place",point_of_interest,restaurant,accounting,liquor_store,health,meal_takeaway,food,locality,pet_store,sublocality_level_1
2,"CN Tower, King and Spadina, Railway Lands, Har...",point_of_interest,airport,travel_agency,university,locality,health,restaurant,cafe,sublocality_level_1,transit_station
3,Central Bay Street,point_of_interest,lodging,restaurant,bank,locality,electronics_store,neighborhood,supermarket,bar,embassy
4,Christie,point_of_interest,lawyer,grocery_or_supermarket,health,hardware_store,sublocality_level_1,gym,locality,car_dealer,food


In [182]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([4, 0, 3, 1, 0, 1, 2, 3, 3, 3], dtype=int32)

In [183]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = Toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3,point_of_interest,electronics_store,car_dealer,bar,meal_delivery,finance,neighborhood,bakery,locality,car_rental
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,lodging,restaurant,university,shopping_mall,clothing_store,point_of_interest,meal_delivery,locality,electronics_store,neighborhood
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,2,lodging,point_of_interest,local_government_office,electronics_store,neighborhood,movie_theater,restaurant,locality,department_store,hospital
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,point_of_interest,electronics_store,dentist,health,locality,general_contractor,home_goods_store,insurance_agency,sublocality_level_1,bicycle_store
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,4,restaurant,bar,lodging,night_club,neighborhood,tourist_attraction,museum,travel_agency,stadium,point_of_interest


In [186]:
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[toronto_merged['Latitude'].mean(), toronto_merged['Longitude'].mean()], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], 
                                  toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [187]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,East Toronto,0,point_of_interest,electronics_store,dentist,health,locality,general_contractor,home_goods_store,insurance_agency,sublocality_level_1,bicycle_store
6,Downtown Toronto,0,point_of_interest,lawyer,grocery_or_supermarket,health,hardware_store,sublocality_level_1,gym,locality,car_dealer,food
9,East York/East Toronto,0,point_of_interest,general_contractor,jewelry_store,electronics_store,sublocality_level_1,church,locality,gas_station,finance,secondary_school
14,West Toronto,0,point_of_interest,restaurant,accounting,liquor_store,health,meal_takeaway,food,locality,pet_store,sublocality_level_1
15,East Toronto,0,point_of_interest,accounting,restaurant,home_goods_store,locality,movie_theater,pet_store,police,health,sublocality_level_1
19,Central Toronto,0,point_of_interest,general_contractor,painter,school,doctor,sublocality_level_1,electronics_store,primary_school,locality,electrician
25,West Toronto,0,point_of_interest,real_estate_agency,insurance_agency,health,sublocality_level_1,bank,general_contractor,movie_theater,locality,finance
29,Central Toronto,0,point_of_interest,health,primary_school,hospital,insurance_agency,sublocality_level_1,store,finance,beauty_salon,school
31,Central Toronto,0,point_of_interest,meal_delivery,embassy,health,sublocality_level_1,bank,finance,beauty_salon,locality,primary_school


In [188]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Downtown Toronto,1,lodging,restaurant,university,shopping_mall,clothing_store,point_of_interest,meal_delivery,locality,electronics_store,neighborhood
5,Downtown Toronto,1,point_of_interest,lodging,restaurant,bank,locality,electronics_store,neighborhood,supermarket,bar,embassy
10,Downtown Toronto,1,lodging,bar,point_of_interest,tourist_attraction,restaurant,neighborhood,gym,supermarket,art_gallery,stadium
27,Downtown Toronto,1,lodging,point_of_interest,university,library,school,liquor_store,locality,neighborhood,church,restaurant
30,Downtown Toronto,1,lodging,point_of_interest,locality,neighborhood,gym,travel_agency,beauty_salon,liquor_store,book_store,electronics_store
34,Downtown Toronto Stn A,1,lodging,restaurant,point_of_interest,bar,shoe_store,gym,tourist_attraction,neighborhood,movie_theater,locality
35,Downtown Toronto,1,food,pet_store,lodging,restaurant,meal_delivery,point_of_interest,real_estate_agency,locality,local_government_office,gas_station
37,Downtown Toronto,1,lodging,point_of_interest,restaurant,hair_care,gym,electronics_store,neighborhood,locality,meal_delivery,florist


In [190]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,2,lodging,point_of_interest,local_government_office,electronics_store,neighborhood,movie_theater,restaurant,locality,department_store,hospital
7,Downtown Toronto,2,lodging,restaurant,locality,hospital,night_club,department_store,bank,bar,neighborhood,electronics_store
13,Downtown Toronto,2,lodging,bar,restaurant,hospital,neighborhood,bank,locality,furniture_store,embassy,finance
16,Downtown Toronto,2,lodging,bar,restaurant,point_of_interest,neighborhood,locality,hospital,department_store,general_contractor,gas_station
36,Downtown Toronto,2,lodging,restaurant,bar,department_store,locality,meal_delivery,neighborhood,hospital,bank,bicycle_store
